## Importing libraries

In [1]:
from math import ceil
from typing import Dict, List, Optional, Union

import cirq
import numpy as np

## Creating classical data

In [2]:
is_low_rank_assumption_passed = False
N_TRIES_LIMIT = 10

i = 0
while not is_low_rank_assumption_passed and i <= N_TRIES_LIMIT:
    i += 1
    # Number of users
    m = 16
    # Number of products
    n = 8
    # Preference matrix
    P = np.random.randint(2, size=(m, n))
    
    # Rank of approximation
    k = 5
    
    U, S, V = np.linalg.svd(P)
    S[k:] = 0
    rank = np.sum(S > 0)
    is_low_rank_assumption_passed = rank == k
    if is_low_rank_assumption_passed and [0, 0, 0, 0, 0, 0, 0, 0] not in P:
        reconstruct_S = np.zeros((U.shape[0], V.shape[0]))
        np.fill_diagonal(reconstruct_S, S)
        approximation_matrix = U @ reconstruct_S @ V
        epsilon = np.linalg.norm(P - approximation_matrix) / np.linalg.norm(P)

if i == N_TRIES_LIMIT + 1:
    raise RuntimeError("Couldn't find an approximation matrix with the given parameters.")

print(f"P = {P}")

P = [[1 0 1 1 1 0 1 1]
 [0 0 1 1 0 1 1 0]
 [0 1 0 1 0 0 0 0]
 [0 1 0 1 1 0 0 1]
 [0 0 1 1 1 1 1 0]
 [1 1 0 1 0 0 1 1]
 [0 1 0 0 1 0 0 0]
 [1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1]
 [0 1 0 1 0 1 1 1]
 [0 0 1 1 0 0 1 1]
 [1 0 1 1 0 0 1 1]
 [1 0 1 0 1 1 0 0]
 [1 1 1 0 1 1 1 0]
 [0 0 1 0 1 1 0 1]
 [0 1 0 0 1 1 1 1]]


## Creating data structure

In [ ]:
class _Leaf:
    def __init__(self, value: float, positive_sign: bool):
        self.value: float = value
        self.positive_sign: bool = positive_sign
    
    @property
    def is_leaf(self):
        return True

class _BinaryTree:
    def __init__(self, depth: int, value: float = 0.):
        self.value: float = value
        self.depth: int = depth
        self.left: Union[_BinaryTree, _Leaf] = _Leaf(0, True)
        self.right: Union[_BinaryTree, _Leaf] = _Leaf(0, True)
    
    def __getitem__(self, key: int):      
        if key - 1 < 0:
            raise KeyError(f"Invalid key: {key} (must be strictly positive).")
            
        binary_path: str = bin(key - 1)[2:]
        binary_path = '0' * (self.depth - len(binary_path)) + binary_path
            
        if len(binary_path) != self.depth:
            raise KeyError(f"Invalid key for storage: {position} (too long).")
        
        def get_aux(binary_tree: _BinaryTree, binary_path: str):
            if len(binary_path) == 0:
                raise ValueError(f"Invalid value encountered for binary_path: nil length.")
            
            position: str = binary_path[0]
            
            if len(binary_path) == 1:
                if position == "0":
                    if binary_tree.left.value == -1:
                        raise KeyError()
                    return binary_tree.left.value, binary_tree.left.positive_sign
                if binary_tree.right.value == -1:
                    raise KeyError()
                return binary_tree.right.value, binary_tree.right.positive_sign

            if position == "0":
                if isinstance(binary_tree.left, _Leaf):
                    raise KeyError()
                
                return get_aux(binary_tree.left, binary_path[1:])
            else:
                if isinstance(binary_tree.right, _Leaf):
                    raise KeyError()
                
                return get_aux(binary_tree.right, binary_path[1:])
        
        try:
            return get_aux(self, binary_path)
        except KeyError:
            raise KeyError(f"No value stored for key: {key}.")
    
    def __str__(self):
        return f"_BinaryTree(depth={self.depth}, value={self.value:.2f})"
    
    @property
    def is_leaf(self) -> bool:
        return False
    
    def store(self, position: int, value: float, positive_sign: bool) -> None:
        def store_aux(binary_tree: _BinaryTree, binary_path: str, value: float, positive_sign: bool) -> None:
            if len(binary_path) == 0:
                raise ValueError(f"Invalid value encountered for binary_path: nil length.")
            
            position: str = binary_path[0]
            binary_tree.value += value
            
            if len(binary_path) == 1:
                if position == "0":
                    binary_tree.left = _Leaf(value, positive_sign)
                else:
                    binary_tree.right = _Leaf(value, positive_sign)
            else:
                if position == "0":
                    if binary_tree.left.is_leaf:
                        binary_tree.left = _BinaryTree(len(binary_path) - 1)
                        
                    store_aux(binary_tree.left, binary_path[1:], value, positive_sign)
                else:
                    if binary_tree.right.is_leaf:
                        binary_tree.right = _BinaryTree(len(binary_path) - 1)
                        
                    store_aux(binary_tree.right, binary_path[1:], value, positive_sign)
        
        if isinstance(position, int):
            if position - 1 < 0:
                raise ValueError(f"Invalid position: {position} (must be strictly positive).")
                
            binary_path = bin(position - 1)[2:]
            binary_path = '0' * (self.depth - len(binary_path)) + binary_path
            
            if len(binary_path) != self.depth:
                raise KeyError(f"Invalid key for storage: {position} (too long).")
            
            store_aux(self, binary_path, value, positive_sign)
        else:
            raise TypeError(f"Incorrect type for position: {type(position)} (int expected).")
            

class QRAM:
    def __init__(self, array: np.ndarray):
        if not isinstance(array, np.ndarray):
            raise TypeError(f"Can only store np.ndarray ({type(array)} given).")
        if len(array.shape) > 2:
            raise ValueError(f"Can't store arrays with more than 2 dimensions (shape {array.shape} given).")
        if len(array.shape) == 1:
            array = array.reshape((1, array.shape[0]))

        self.m: int = array.shape[0]
        self.n: int = array.shape[1]
        n_binary_writing: str = bin(self.n)[3:]
        self.trees_depth: int = len(n_binary_writing) + 1 if "1" in n_binary_writing else len(n_binary_writing)
        m_binary_writing: str = bin(self.m)[3:]
        self.log2m: int = len(m_binary_writing) + 1 if "1" in m_binary_writing else len(m_binary_writing)
        self._trees: Dict[int, _BinaryTree] = {}
        norms: np.ndarray = np.linalg.norm(array, axis=1)
            
        if self.m == 1:
            self._store_init(array / norms)
            self.qram_norms: Optional[QRAM] = None
        else:
            self._store_init(array / norms.reshape((norms.shape[0], 1)))
            self.qram_norms: Optional[QRAM] = QRAM(norms)
            
    
    def __getitem__(self, key):
        if isinstance(key, int):
            return self._trees[key]
        if isinstance(key, tuple):
            if len(key) != 2:
                raise KeyError(f"An address is made of 2 indexes ({len(tuple)} given).")
            if self._trees.get(key[0]) is None:
                self._trees[i] = _BinaryTree(self.trees_depth)
                
            return self._trees[key[0]][key[1]]
        raise KeyError(f"Not a representable address: {key}.")
    
    def __repr__(self):
        return f"QRAM({[str(self._trees[x]) for x in self._trees]})"
    
    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            if len(key) != 2:
                raise KeyError(f"An address is made of 2 indexes ({len(key)} given).")
            if not (isinstance(value, tuple) or isinstance(value, float) or isinstance(value, np.int64) or isinstance(value, np.float64)):
                raise TypeError(f"Not an acceptable type to store: {type(value)} (numeric type or tuple expected).")
            if isinstance(value, float) or isinstance(value, np.int64) or isinstance(value, np.float64):
                self._store(key[0], key[1], value ** 2, value >= 0)
                return
            if len(value) != 2:
                raise ValueError(f"value must be a tuple of length 2 (length {len(value)} given).")
            if not isinstance(value[0], float):
                raise ValueError(f"The first element of value must be a float ({type(value[0])} given).")
            if value[0] < 0 or value[0] > 1:
                raise ValueError(f"The first element of value must be between 0 and 1 ({value[0]} given).")
            if not isinstance(value[1], bool):
                raise ValueError(f"The second element of value must be a boolean ({type(value[1])} given).")
                
            self._store(key[0], key[1], value[0], value[1])
        else:
            raise TypeError(f"Invalid address given: {key}.")
    
    def _store(self, i: int, j: int, value: float, positive_sign: bool) -> None:
        if self._trees.get(i) is None:
            self._trees[i] = _BinaryTree(self.trees_depth)
        
        self._trees[i].store(j, value, positive_sign)
    
    def _store_init(self, array: np.ndarray) -> None:        
        for i in range(array.shape[0]):
            for j in range(array.shape[1]):
                self[i + 1, j + 1] = array[i, j]

## Creating gates

In [3]:
# TODO: Parallelize execution
class _LoadingVectorPowGate(cirq.Gate):
    def __init__(self, qram: QRAM, index: int, exponent: int = 1.):
        self.qram: QRAM = qram
        self.index: int = index
        # Used to invert the gate
        if exponent not in [-1, 0, 1]:
            raise ValueError(f"_LoadingVectorGate can only be raised to power 1, 0 or -1 (exponent {exponent} given).")
        self.exponent: int = exponent
        cirq.Gate.__init__(self)
    
    def _num_qubits_(self) -> int:
        return self.qram.trees_depth
    
    def _decompose_(self, qubits):
        def _decompose_aux(tree, qubits, binary_writing, operations):
            if not tree.value:
                return
            depth: int = len(binary_writing)
            theta: float = np.arccos(np.sqrt(tree.left.value / tree.value)) + np.arcsin(np.sqrt(tree.right.value / tree.value))
            operations.append((cirq.ry(theta) ** self.exponent).on(qubits[depth]).controlled_by(*qubits[:depth], control_values=binary_writing))
            
            if tree.left.is_leaf and tree.right.is_leaf:
                if tree.left.positive_sign and tree.right.positive_sign:
                    return
                elif tree.left.positive_sign:
                    operations.append((cirq.Z ** self.exponent).on(qubits[depth]).controlled_by(*qubits[:depth], control_values=binary_writing)) 
                    return
                elif tree.right.positive_sign:
                    operations.append((cirq.Z ** self.exponent).on(qubits[depth]).controlled_by(*qubits[:depth], control_values=binary_writing)) 
                    
                operations.append((cirq.ry(2 * np.pi) ** self.exponent).on(qubits[depth]).controlled_by(*qubits[:depth], control_values=binary_writing)) 
                return
            elif tree.left.is_leaf:
                _decompose_aux(tree.right, qubits, binary_writing + [1], operations)
                return
            elif tree.right.is_leaf:
                _decompose_aux(tree.left, qubits, binary_writing + [0], operations)
                return
            
            _decompose_aux(tree.left, qubits, binary_writing + [0], operations)
            _decompose_aux(tree.right, qubits, binary_writing + [1], operations)
        
        res = []
        _decompose_aux(self.qram._trees[self.index], qubits[:self.qram.trees_depth], [], res)
        
        return res if self.exponent >= 0 else res[::-1]
    
    def __pow__(self, exponent, modulo=None):
        return _LoadingVectorPowGate(self.qram, self.index, exponent)
    
    def __repr__(self):
        return f"LV({self.index}) ** {self.exponent}"

    
class LoadingVectorGate(_LoadingVectorPowGate):
    def __init__(self, qram: QRAM, index: int):
        _LoadingVectorPowGate.__init__(self, qram, index, 1.)
    
    def __repr__(self):
        return f"LV({self.index})"


# TODO: Parallelize execution
class _LoadingPowGate(cirq.Gate):
    def __init__(self, qram: QRAM, exponent: int):
        self.qram: QRAM = qram
        
        if exponent not in [-1, 0, 1]:
            raise ValueError(f"_LoadingPowGate can only be raised to power 1, 0 or -1 (exponent {exponent} given).")
        
        self.exponent: int = exponent
    
    def _num_qubits_(self) -> int:
        return self.qram.trees_depth + self.qram.log2m
    
    def _decompose_(self, qubits):
        operations = []
        control_qubits, state_qubits = qubits[:self.qram.log2m], qubits[self.qram.log2m:]
        
        for i in range(2 ** self.qram.log2m):
            control_values = bin(i)[2:]
            control_values = "0" * (len(control_qubits) - len(control_values)) + control_values
            control_values = [int(c) for c in control_values]
            operations.append((LoadingVectorGate(self.qram, i + 1) ** self.exponent).on(*state_qubits).controlled_by(*control_qubits, control_values=control_values))
        
        return operations if self.exponent >= 0 else operations[::-1]
    
    def __pow__(self, exponent, modulo=None):
        return _LoadingPowGate(self.qram, exponent)
    
    def __repr__(self):
        return f"LG ** {self.exponent}"


class LoadingGate(_LoadingPowGate):
    def __init__(self, qram: QRAM):
        _LoadingPowGate.__init__(self, qram, 1)
    
    def __repr__(self):
        return "LG"


class NormGate(LoadingVectorGate):
    def __init__(self, qram: QRAM):
        LoadingVectorGate.__init__(self, qram.qram_norms, 1)
    
    def __repr__(self):
        return "N"

class _UVPowGate(cirq.Gate):
    def __init__(self, qram: QRAM, exponent: int):
        if not isinstance(exponent, int):
            raise TypeError(f"UVGate can't be raised to a non-integer exponent (exponent {exponent} given)")
        self.exponent: int = exponent
        self.qram: QRAM = qram
        cirq.Gate.__init__(self)
    
    def _num_qubits_(self) -> int:
        return self.qram.trees_depth + self.qram.log2m
    
    def _decompose_(self, qubits):
        operations = []
        first_register, second_register = qubits[:self.qram.log2m], qubits[self.qram.log2m:]
        
        for _ in range(abs(exponent)):
            if exponent >= 0:
                # Apply V
                operations.append((NormGate(self.qram) ** - 1).on(*first_register))
                operations.append((cirq.IdentityGate() * -1).on(*second_register).controlled_by(*first_register, control_values=[0] * self.qram.log2m))
                operations.append(NormGate(self.qram).on(*first_register))
                
            # Apply U
            operations.append(LoadingGate(self.qram).on(*qubits) ** -1)
            operations.append((cirq.IdentityGate() * -1).on(*first_register).controlled_by(*second_register, control_values=[0] * self.qram.trees_depth))
            operations.append(LoadingGate(self.qram).on(*qubits))
            
            if exponent < 0:
                # Apply V
                operations.append((NormGate(self.qram) ** - 1).on(*first_register))
                operations.append((cirq.IdentityGate() * -1).on(*second_register).controlled_by(*first_register, control_values=[0] * self.qram.log2m))
                operations.append(NormGate(self.qram).on(*first_register))
        
        return operations
    
    def __repr__(self):
        return f"UV ** {self.exponent}"
    
    def __pow__(self, exponent, modulo=None):
        return _UVPowGate(self.qram, exponent)


class UVGate(_UVPowGate):
    def __init__(self, qram):
        _UVPowGate.__init__(self, qram, 1)
    
    def __repr__(self) -> str:
        return "UV"


class _QPEPowGate(cirq.Gate):
    def __init__(self, gate: cirq.Gate, epsilon: float, exponent: int):
        if exponent not in [-1, 0, 1]:
            raise ValueError(f"QPEGate can only be raised to power 1, 0 or -1 (exponent {exponent} given).")
        self.exponent: int = exponent
        self.gate: cirq.Gate = gate
        self.t: int = ceil(np.log2(1 / epsilon))
        self.probability: float = 1 - 1 / (2 * (2 ** additional_qubits - 2))
        cirq.Gate.__init__(self)
    
    def _num_qubits_(self) -> int:
        return self.gate._num_qubits_() + self.t
    
    def _decompose_(self, qubits):
        operations = []
        first_register, second_register = qubits[:self.t], qubits[self.t:]
        operations.append(cirq.H.on_each(*first_register))
        operations += [self.gate.on(*second_register).controlled_by(first_register[i]) ** ((2 ** i) * self.exponent) for i in range(self.t)]
        operations.append(cirq.QFT(inverse=True).on(*first_register) ** self.exponent)
        
        return operations if self.exponent >= 0 else operations[::-1]
    
    def __pow__(self, exponent, modulo=None):
        return _QPEPowGate(self.gate, self.epsilon, exponent)
    
    def __repr__(self):
        return f"QPE({self.probability:.3f})"


class QPEGate(_QPEPowGate):
    def __init__(self, gate: cirq.Gate, epsilon: float):
        _QPEPowGate.__init__(self, gate, epsilon, 1)
    
    def __repr__(self) -> int:
        return "QPE"
    

class _QSVEPowGate(cirq.Gate):
    def __init__(qram_A: QRAM, epsilon: float, exponent: int):
        if exponent not in [-1, 0, 1]:
            raise ValueError(f"QSVEGate can only be raised to power 1, 0 or -1 (exponent {exponent} given).")
        self.exponent: int = exponent
        self.qram_A: QRAM = qram_A
        self.epsilon: float = epsilon
        self.qpe_gate: QPEGate = QPEGate(UVGate(self.qram_A), 2 * epsilon)
        cirq.Gate.__init__(self)
        
    def _num_qubits_(self) -> int:
        return self.qram_A.log2m + self.qram_A.trees_depth + self.qpe_gate.t
    
    def _decompose_(self, qubits):
        operations = []
        first_register = qubits[:self.qram_A.log2m]
        second_register = qubits[self.qram_A.log2m:self.qram_A.log2m + self.qram_A.trees_depth]
        third_register = qubits[self.qram_A.log2m + self.qram_A.trees_depth:]
        operations.append(NormGate(qram_A).on(*first_register) ** self.exponent)
        operations.append(self.qpe_gate.on(*qubits) ** self.exponent)
        operations.append((NormGate(qram_A) ** -1).on(*first_register) ** self.exponent)
        
        return operations if self.exponent >= 0 else operations[::-1]
    
    def __pow__(self, exponent, modulo=None):
        return _QSVEPowGate(self.qram_A, self.epsilon, exponent)
    
    def __repr__(self) -> str:
        return f"QSVE ** {self.exponent}"


class QSVEGate(_QSVEPowGate):
    def __init__(self, qram_A: QRAM, epsilon: float):
        _QSVEPowGate.__init__(self, qram_A, epsilon, 1)
    
    def __repr__(self) -> str:
        return "QSVE"

    
class ThresholdGate(cirq.Gate):
    def __init__(self, throshold: float, num_qubits: int):
        self.threshold: float = threshold
        self.bin_threshold: List[int] = [int(x) for x in bin(threshold)[2:]]
        self.bin_threshold = [0] * (num_qubits - len(self.bin_threshold) - 1) + self.bin_threshold
        cirq.Gate.__init__(self)
    
    def _num_qubits_() -> int:
        return len(self.bin_threshold) + 1
    
    def _decompose_(self, qubits):
        operations = []
        input_qubits = qubits[:-1]
        res_qubit = qubits[-1]
        

In [4]:
simulator = cirq.Simulator()
qubits = [cirq.GridQubit(1, i) for i in range(2)]
circuit = cirq.Circuit()
circuit.append(cirq.H.on_each(*qubits))
circuit.append(cirq.measure(*qubits, key='x'))
circuit.append(cirq.H.on_each(*qubits))
circuit.append(cirq.measure(*qubits, key='y'))
res = simulator.run(circuit, repetitions=5)
d = res.histogram(key='x')
print(circuit)
print(d.most_common()[0])
print(res)
print(d)
res.measurements['x'][-1]

(1, 0): ───H───M('x')───H───M('y')───
               │            │
(1, 1): ───H───M────────H───M────────
(2, 2)
x=01111, 00101
y=10110, 10111
Counter({2: 2, 3: 2, 0: 1})


array([1, 1], dtype=uint8)

In [13]:
N = 6
phi = np.random.random(N) - .5

ds_phi = QRAM(phi)
circuit = cirq.Circuit()
qubits = [cirq.GridQubit(1, i) for i in range(ds_phi.trees_depth)]
circuit.append(LoadingVectorGate(ds_phi, 1).on(*qubits))
simulator = cirq.Simulator()
result = simulator.simulate(circuit)
print(np.linalg.norm(np.hstack((phi, np.zeros(result.final_state.shape[0] - N))) / np.linalg.norm(phi) - result.final_state))
print(np.hstack((phi, np.zeros(result.final_state.shape[0] - N))) / np.linalg.norm(phi))
print(np.real(result.final_state))
print('\n' + '=' * 115 + '\n')
print(circuit)
visualization_circuit = cirq.Circuit()
visualization_circuit.append(cirq.decompose(circuit[0].operations[0]))
print('\n' + '=' * 115 + '\n')
print(visualization_circuit)

4.9818998766071636e-08
[ 0.77394034 -0.47372964 -0.03368434  0.3088555   0.28119602  0.03160729
  0.          0.        ]
[ 0.7739404  -0.47372964 -0.03368434  0.3088555   0.281196    0.03160729
  0.          0.        ]


(1, 0): ───LV(1)───
           │
(1, 1): ───#2──────
           │
(1, 2): ───#3──────


(1, 0): ───Ry(0.183π)───(0)─────────(0)─────────(0)───(0)──────────(0)───(0)────────@──────────@────────────
                        │           │           │     │            │     │          │          │
(1, 1): ────────────────Ry(0.21π)───(0)─────────(0)───@────────────@─────@──────────Ry(0.0π)───(0)──────────
                                    │           │     │            │     │                     │
(1, 2): ────────────────────────────Ry(0.35π)───Z─────Ry(0.931π)───Z─────Ry(2.0π)──────────────Ry(0.071π)───


In [6]:
def quantum_sve(qram_A: QRAM, qram_x: QRAM, epsilon: float, circuit: cirq.Circuit, qubits: List[cirq.GridQubit]) -> None:
    # 1. Create state
    state_qubits = qubits[qram_A.log2m:qram_A.log2m + qram_x.trees_depth]
    circuit.append(LoadingVectorGate(qram_x, 1).on(*state_qubits))
    # 2. Apply Q
    circuit.append(NormGate(qram_A).on(*qubits[:qram_A.log2m]))
    # 3. Apply QPE
    circuit.append(QPEGate(UVGate(qram_A), 2 * epsilon))
    # 4. Compute singular values
    # 5. Uncompute 2.
    circuit.append((NormGate(qram_A) ** -1).on(*qubits[:qram_A.log2m]))

    

In [7]:
circuit = cirq.Circuit()
qram_P = QRAM(P)
qubits = [cirq.GridQubit(0, i) for i in range(qram_P.log2m + qram_P.trees_depth)]
circuit.append(cirq.H.on_each(*qubits[:4]))
circuit.append(LoadingGate(qram_P).on(*qubits))
result = simulator.simulate(circuit)
print(np.real(result.final_state).reshape(P.shape))
print(P / np.linalg.norm(P, axis=1).reshape(16, 1))
print(circuit)

[[0.10206206 0.         0.10206207 0.10206207 0.10206206 0.
  0.10206207 0.10206207]
 [0.         0.         0.12499999 0.12499999 0.         0.12499999
  0.12499999 0.        ]
 [0.         0.17677668 0.         0.17677668 0.         0.
  0.         0.        ]
 [0.         0.12499999 0.         0.12499999 0.12499999 0.
  0.         0.12499999]
 [0.         0.         0.11180338 0.11180338 0.11180338 0.11180338
  0.11180338 0.        ]
 [0.11180338 0.11180338 0.         0.11180338 0.         0.
  0.11180338 0.11180338]
 [0.         0.17677668 0.         0.         0.17677668 0.
  0.         0.        ]
 [0.08838834 0.08838834 0.08838834 0.08838834 0.08838834 0.08838834
  0.08838834 0.08838834]
 [0.         0.09449111 0.09449112 0.09449112 0.09449109 0.09449109
  0.09449109 0.09449109]
 [0.         0.11180338 0.         0.11180338 0.         0.11180338
  0.11180338 0.11180338]
 [0.         0.         0.12499999 0.12499999 0.         0.
  0.12499999 0.12499999]
 [0.11180338 0.         0

In [10]:
circuit = cirq.Circuit()
qram_phi = QRAM(phi)
qubits = [cirq.GridQubit(0, i) for i in range(qram_phi.trees_depth)]
circuit.append(LoadingVectorGate(qram_phi, 1).on(*qubits))
circuit.append((LoadingVectorGate(qram_phi, 1) ** -1).on(*qubits))
result = simulator.simulate(circuit)
print(circuit)
visualization_circuit = cirq.Circuit()
visualization_circuit.append(cirq.decompose(circuit[0].operations[0]))
print('\n' + '=' * 115 + '\n')
print(visualization_circuit)
visualization_circuit = cirq.Circuit()
visualization_circuit.append(cirq.decompose(circuit[1].operations[0]))
print('\n' + '=' * 115 + '\n')
print(visualization_circuit)
print('\n' + '=' * 115 + '\n')
print(result.final_state)
expected = np.zeros(N)
expected[0] = 1
expected = np.hstack((expected, np.zeros(result.final_state.shape[0] - N))) / np.linalg.norm(expected)
print(np.linalg.norm(expected-result.final_state))

(0, 0): ───LV(1)───LV(1) ** -1───
           │       │
(0, 1): ───#2──────#2────────────
           │       │
(0, 2): ───#3──────#3────────────


(0, 0): ───Ry(0.386π)───(0)──────────(0)─────────(0)───(0)──────────(0)────────@──────────@────────────
                        │            │           │     │            │          │          │
(0, 1): ────────────────Ry(0.452π)───(0)─────────(0)───@────────────@──────────Ry(0.0π)───(0)──────────
                                     │           │     │            │                     │
(0, 2): ─────────────────────────────Ry(0.03π)───Z─────Ry(0.117π)───Ry(2.0π)──────────────Ry(0.581π)───


(0, 0): ───@─────────────@───────────(0)────────(0)───────────(0)───(0)──────────(0)───────────Ry(-0.386π)───
           │             │           │          │             │     │            │
(0, 1): ───(0)───────────Ry(-0.0π)───@──────────@─────────────(0)───(0)──────────Ry(-0.452π)─────────────────
           │                         │          │    

In [9]:
qubits = [cirq.LineQubit(i) for i in range(5)]
backup_res = []
simulator = cirq.Simulator()
main_simulator = cirq.Simulator()
circuit = cirq.Circuit()
main_circuit = cirq.Circuit()
main_circuit.append(cirq.X.on(qubits[2]))
main_circuit.append(cirq.X.on(qubits[4]))
circuit.append(cirq.H.on_each(*qubits[1:]))
circuit.append(cirq.measure(*qubits, key='x'))
results = simulator.run(circuit)
print(circuit)
results.measurements['x']

0: ───────M('x')───
          │
1: ───H───M────────
          │
2: ───H───M────────
          │
3: ───H───M────────
          │
4: ───H───M────────


array([[0, 1, 1, 0, 0]], dtype=uint8)